<a href="https://colab.research.google.com/github/ahmedalali-do/fsdl-project/blob/main/AutoNLU_Mobile_Banking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mobile Banking Aspect-based Sentiment Analysis AutoNLU

## Setup & Installation

#### Installing Dependencies

In [ ]:
%env PIP_PULL=
! pip install autonlu --extra-index-url=https://${PIP_PULL}:@pypi.fury.io/deepopinion

#### Define Environment Variiables

In [ ]:
%load_ext tensorboard

!pip install xmltodict -q

import autonlu
from autonlu import Model

import pandas as pd
import numpy as np

import requests
import xmltodict

In [ ]:
#login to your account and test that it is connected
autonlu.login()  # Will prompt for user and password

model = autonlu.Model("DeepOpinion/hotels_absa_en")
model.predict(["The room was nice, but the staff was unfriendly"])
# Returns [[['Room', 'POS'], ['Staff', 'NEG']]]

### Listing Models & Projects

In [ ]:
#all availablepretrained models
from autonlu import Model, list_models

In [ ]:
# List all projects of user account
autonlu.list_models()

## Train A Model

### Upload Training Data

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
data = pd.read_csv('mobile_banking_training_data.csv').reset_index()

In [ ]:
data.head()

In [ ]:
#merge aspect & sentiment labels to a list

data['absa'] = data[['aspect','sentiment']].values.tolist()

In [ ]:
#aggregate labels for multi-label classification

train = data.groupby('segment_text')['absa'].apply(list).reset_index(name='absa')[:175]

In [ ]:
train

### Upload datasets

### Model Training

In [ ]:
!nvidia-smi

In [ ]:
#confirm that runtime is using GPU 'cuda'
model.device

In [ ]:
#import libraries
%load_ext tensorboard
import gdown

In [ ]:
model = Model("DeepOpinion/banking_base_en", standard_label="neutral", )
model.train(train['segment_text'].to_list(), train['absa'].to_list(), verbose=True)
model.save("mobile_banking_absa_fsdl")

In [ ]:
#import tensorboard for visualization
%tensorboard --logdir tensorboard_logs

In [ ]:
#test model
prediction_model = Model("mobile_banking_absa_fsdl")
ret = prediction_model.predict([
    "I have bank with this over 20 years for one purpose the bill pay system was great. Made my life easy.",
    "I tried to set up auto pay to automatically pay the whole balance every month. No can do. The app automatically schedules the minimum payment every month. No way to change on the app. No help - so far - from customer service. Sad."])
print(ret)